In [57]:
print("Hello World")

Hello World


In [84]:
from pulp import *
from treelib import Node, Tree
import math
import random

tipo = 'LpMaximize'

Para este ejemplo se usa el ejercicio 2 de branch and bound en el cuaderno. 21 de Mayo

In [85]:
problema = LpProblem("Branch_and_Bound", LpMaximize)
lista_variables = []

In [86]:
coeficientes = {'x1': 2, 'x2': 3}
costosRes1 = {'x1': -3, 'x2': 1}
costosRes2 = {'x1': 4, 'x2': 2}
costosRes3 = {'x1': 4, 'x2': -1}
costosRes4 = {'x1': -1, 'x2': 2}

xs = ['x1', 'x2']
x_vars = pulp.LpVariable.dicts("Xs", xs, 0)

problema += pulp.lpSum([coeficientes[i] * x_vars[i] for i in xs]), 'obj' # funcion objetivo

In [87]:
x_vars['x1']

Xs_x1

In [88]:
list(x_vars.values())

[Xs_x1, Xs_x2]

In [89]:
problema += pulp.lpSum([costosRes1[i] * x_vars[i] for i in xs]) <= 1
problema += pulp.lpSum([costosRes2[i] * x_vars[i] for i in xs]) <= 15
problema += pulp.lpSum([costosRes3[i] * x_vars[i] for i in xs]) <= 10
problema += pulp.lpSum([costosRes4[i] * x_vars[i] for i in xs]) <= 5

In [90]:
problema.solve()

1

In [91]:
for variable in problema.variables():
        print(variable.name, "=", variable.varValue)

Xs_x1 = 2.0
Xs_x2 = 3.5


In [92]:
print(pulp.value(problema.objective)) 

14.5


In [93]:
print("Estatus: ", LpStatus[problema.status])

Estatus:  Optimal


In [94]:
if LpStatus[problema.status] != 'Optimal':
    print("No se podrá resolver")

## Arbol

In [95]:
arbol = Tree()
resultado = {}

resultado['obj'] = value(problema.objective) # Valor de z*

In [96]:
pila_nodos = [] # nombres de los nodos (array de strings)
mejor_nodo = {} # Mejor nodo cada tiempo
mejor_nodo['nodo'] = 'none'
mejor_nodo['z']=-float('inf') # -infinito

variables_no_enteras = [] # variables no enteras del problema actual (array de strings)
solucion = True


In [97]:
resultado['vars'] = {}

for v in problema.variables():
    resultado['vars'][v.name] = '{var}={value}'.format(var=v.name, value=v.varValue)
    resultado[v.name] = v.varValue
    
    # is_integer() devuelve True si en el flotante sus decimales son 0
    if not resultado[v.name].is_integer():
        print(v.name, " no es entera")
        solucion = False
        variables_no_enteras.append(v.name)
        
print("Diccionario resultado: ", resultado)
print("Lista de variables no enteras: ", variables_no_enteras)

Xs_x2  no es entera
Diccionario resultado:  {'obj': 14.5, 'vars': {'Xs_x1': 'Xs_x1=2.0', 'Xs_x2': 'Xs_x2=3.5'}, 'Xs_x1': 2.0, 'Xs_x2': 3.5}
Lista de variables no enteras:  ['Xs_x2']


In [98]:
if solucion == True:
    print('Automaticamente entero')
    mejor_nodo['nodo'] = 'original'
    mejor_nodo['z'] = value(problema.objective)
    mejor_nodo['resultado'] = resultado
else: # Agregamos el root a la lista de nodos
    print("Se realizará arbol de decision")
    pila_nodos.append('original')

Se realizará arbol de decision


In [99]:
pila_nodos

['original']

In [100]:
# El primero es un nombre para cuando se imprima, el segundo es un identificador,
# el tercer parametro son los datos que gaurdará
arbol.create_node('Original', 'original',
                 data={
                     'status': LpStatus[problema.status],
                     'problema': problema,
                     'no_enteros': variables_no_enteras,
                     'resultado': resultado
                 })
p = 0

In [101]:
solution = True

# Para cada nodo creamos otros 2 nodos, con sus nuevas restricciones

# FIFO. Para hacer el branching seleccionamos el ultimo nodo insertado en la pila
nodo_ramificar = pila_nodos[0] # es un string

# Lo removemos para que sea examinado
pila_nodos.remove(nodo_ramificar) # pila nodos se quedó vacio

variables_no_enteras = arbol.get_node('{nodo}'.format(nodo=nodo_ramificar)).data['no_enteros']

variable_ramificar = random.choice(variables_no_enteras)
print("Variable por la que se va a ramificar: ", variable_ramificar)

# Aqui 3.5 pasa a 3
piso = int(arbol.get_node('{nodo}'.format(nodo=nodo_ramificar)).data['resultado'][variable_ramificar])

techo = math.ceil(arbol.get_node('{nodo}'.format(nodo=nodo_ramificar)).data['resultado'][variable_ramificar])

print("Piso: ", piso)
print("Techo: ", techo)

Variable por la que se va a ramificar:  Xs_x2
Piso:  3
Techo:  4


In [102]:
# Copiar el original, porque le vamos a añadir una restriccion
nodo = arbol.get_node('{nodo}'.format(nodo=nodo_ramificar)).data['problema'].copy()

In [103]:
x_elegida = xs[int(variable_ramificar.split('_')[1][1]) - 1] 

# Añadir nueva restriccion
nodo += x_vars[x_elegida] <= piso

nueva_restriccion = '{x}<={piso}'.format(x=x_vars[x_elegida], piso=piso)
print(nodo) # subproblema 1

nodo.solve()

for variable in nodo.variables():
        print(variable.name, "=", variable.varValue)
print(value(nodo.objective)) # z* del subproblema 1

Branch_and_Bound:
MAXIMIZE
2*Xs_x1 + 3*Xs_x2 + 0
SUBJECT TO
_C1: - 3 Xs_x1 + Xs_x2 <= 1

_C2: 4 Xs_x1 + 2 Xs_x2 <= 15

_C3: 4 Xs_x1 - Xs_x2 <= 10

_C4: - Xs_x1 + 2 Xs_x2 <= 5

_C5: Xs_x2 <= 3

VARIABLES
Xs_x1 Continuous
Xs_x2 Continuous

Xs_x1 = 2.25
Xs_x2 = 3.0
13.5


In [104]:
# Volver a inicializar resultado
resultado = {}
resultado['vars'] = {}
variables_no_enteras = []

In [113]:
# Lado izquierdo
if LpStatus[nodo.status] == 'Optimal': # Si
    resultado['obj'] = value(nodo.objective) # 13.5
    
    for v in nodo.variables():
        resultado['vars'][v.name] = '{var}={value}'.format(var=v.name, value=v.varValue)
        resultado[v.name] = v.varValue
        if not resultado[v.name].is_integer(): # Por ej aqui, x1* = 2.25
            solution = False
            variables_no_enteras.append(v.name)
    
    if solution == False: # hay una variable NO entera. Habrá que ramificar
            print("Variables no enteras: ", variables_no_enteras)
            
            if mejor_nodo['nodo'] == 'none': # nuestro caso. Aun no tenemos una solucion
                print("Aun no tenemos una solucion")
                pila_nodos.append('node{no}'.format(no = p))
                print("Pila nodos: ", pila_nodos)
            else: 
                if tipo == 'LpMaximize':
                    if resultado['obj'] > mejor_nodo['z']:
                        pila_nodos.append('node{no}'.format(no=p))
                else:
                    if resultado['obj'] < mejor_nodo['z']:
                        pila_nodos.append('node{no}'.format(no=p))
    else: # todas las variables son enteras (una hoja)
        if tipo == 'LpMaximize':
            if resultado['obj'] > mejor_nodo['z']:
                mejor_nodo['node'] = 'node{no}'.format(no=p)
                mejor_nodo['z'] = resultado['obj']
                mejor_nodo['resultado'] = resultado
        else:
            if resultado['obj'] < mejor_nodo['z']:
                mejor_nodo['node'] = 'node{no}'.format(no=p)
                mejor_nodo['z'] = resultado['obj']
                mejor_nodo['resultado'] = resultado
    
    arbol.create_node('node{no}'.format(no=p), 'node{no}'.format(no=p), 
                      parent='{node}'.format(node=nodo_ramificar),
                     data={
                         'status': LpStatus[nodo.status],
                         'problema': nodo,
                         'no_enteros': variables_no_enteras,
                         'resultado': resultado,
                         'nueva_restriccion': nueva_restriccion
                     })
    
else: # El status NO es optimal
    arbol.create_node('node{no}'.format(no=p), 'node{no}'.format(no=p), 
                      parent='{node}'.format(node=nodo_ramificar),
                     data={
                         'status': LpStatus[nodo.status],
                         'problema': nodo,
                         'no_enteros': variables_no_enteras,
                         'resultado': {'obj':LpStatus[nodo.status]},
                         'nueva_restriccion': nueva_restriccion
                     })
p += 1

In [114]:
mejor_nodo

{'nodo': 'none', 'z': -inf}

In [106]:
arbol.show()

Original
└── node0



In [115]:
# Lado derecho

# Copiar el original, porque le vamos a añadir una restriccion
nodo = arbol.get_node('{nodo}'.format(nodo=nodo_ramificar)).data['problema'].copy()

nodo += x_vars[x_elegida] >= techo

nueva_restriccion = '{x}>={bs}'.format(x = x_vars[x_elegida], bs = techo)

nodo.solve() # Infeasible

-1

In [116]:
# Volver a inicializar resultado
resultado = {}
resultado['vars'] = {}
variables_no_enteras = []

solution = True

In [117]:
if LpStatus[nodo.status] == 'Optimal': # No
    resultado['obj'] = value(nodo.objective) 
    
    for v in nodo.variables():
        resultado['vars'][v.name] = '{var}={value}'.format(var=v.name, value=v.varValue)
        resultado[v.name] = v.varValue
        if not resultado[v.name].is_integer(): 
            solution = False
            variables_no_enteras.append(v.name)
    
    if solution == False: 
            print("Variables no enteras: ", variables_no_enteras)
            
            if mejor_nodo['nodo'] == 'none': 
                print("Aun no tenemos una solucion")
                pila_nodos.append('node{no}'.format(no = p))
                print("Pila nodos: ", pila_nodos)
            else: 
                if tipo == 'LpMaximize':
                    if resultado['obj'] > mejor_nodo['z']:
                        pila_nodos.append('node{no}'.format(no=p))
                else:
                    if resultado['obj'] < mejor_nodo['z']:
                        pila_nodos.append('node{no}'.format(no=p))
    else: # todas las variables son enteras (una hoja)
        if tipo == 'LpMaximize':
            if resultado['obj'] > mejor_nodo['z']:
                mejor_nodo['node'] = 'node{no}'.format(no=p)
                mejor_nodo['z'] = resultado['obj']
                mejor_nodo['resultado'] = resultado
        else:
            if resultado['obj'] < mejor_nodo['z']:
                mejor_nodo['node'] = 'node{no}'.format(no=p)
                mejor_nodo['z'] = resultado['obj']
                mejor_nodo['resultado'] = resultado
    
    arbol.create_node('node{no}'.format(no=p), 'node{no}'.format(no=p), 
                      parent='{node}'.format(node=nodo_ramificar),
                     data={
                         'status': LpStatus[nodo.status],
                         'problema': nodo,
                         'no_enteros': variables_no_enteras,
                         'resultado': resultado,
                         'nueva_restriccion': nueva_restriccion
                     })
    
else: # El status NO es optimal
    arbol.create_node('node{no}'.format(no=p), 'node{no}'.format(no=p), 
                      parent='{node}'.format(node=nodo_ramificar),
                     data={
                         'status': LpStatus[nodo.status],
                         'problema': nodo,
                         'no_enteros': variables_no_enteras,
                         'resultado': {'obj':LpStatus[nodo.status]},
                         'nueva_restriccion': nueva_restriccion
                     })
p += 1

In [118]:
arbol.show()

Original
├── node0
├── node1
├── node2
└── node3



In [119]:
for nodo in arbol.all_nodes():
    print(nodo)

Node(tag=Original, identifier=original, data={'status': 'Optimal', 'problema': Branch_and_Bound:
MAXIMIZE
2*Xs_x1 + 3*Xs_x2 + 0
SUBJECT TO
_C1: - 3 Xs_x1 + Xs_x2 <= 1

_C2: 4 Xs_x1 + 2 Xs_x2 <= 15

_C3: 4 Xs_x1 - Xs_x2 <= 10

_C4: - Xs_x1 + 2 Xs_x2 <= 5

VARIABLES
Xs_x1 Continuous
Xs_x2 Continuous
, 'no_enteros': ['Xs_x2'], 'resultado': {'obj': 14.5, 'vars': {'Xs_x1': 'Xs_x1=2.0', 'Xs_x2': 'Xs_x2=3.5'}, 'Xs_x1': 2.0, 'Xs_x2': 3.5}})
Node(tag=node0, identifier=node0, data={'status': 'Optimal', 'problema': Branch_and_Bound:
MAXIMIZE
2*Xs_x1 + 3*Xs_x2 + 0
SUBJECT TO
_C1: - 3 Xs_x1 + Xs_x2 <= 1

_C2: 4 Xs_x1 + 2 Xs_x2 <= 15

_C3: 4 Xs_x1 - Xs_x2 <= 10

_C4: - Xs_x1 + 2 Xs_x2 <= 5

_C5: Xs_x2 <= 3

VARIABLES
Xs_x1 Continuous
Xs_x2 Continuous
, 'no_enteros': ['Xs_x1'], 'resultado': {'vars': {'Xs_x1': 'Xs_x1=2.25', 'Xs_x2': 'Xs_x2=3.0'}, 'obj': 13.5, 'Xs_x1': 2.25, 'Xs_x2': 3.0}, 'nueva_restriccion': 'Xs_x2<=3'})
Node(tag=node1, identifier=node1, data={'status': 'Infeasible', 'problema': Bran